#Formatting _Siku quanshu_
reading a text file and format it into an xml file

In [5]:
import re
document = open("/Users/pingtzuhu/Documents/佩文韻府/test.txt", encoding="utf8")
newDoc="""<?xml version="1.0" encoding="UTF-8"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml"
	schematypens="http://purl.oclc.org/dsdl/schematron"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0">
  <teiHeader>
      <fileDesc>
         <titleStmt>
            <title>Title</title>
         </titleStmt>
         <publicationStmt>
            <p>Publication Information</p>
         </publicationStmt>
         <sourceDesc>
            <p>Information about the source</p>
         </sourceDesc>
      </fileDesc>
  </teiHeader>
  <text>
      <body>
    """ #newXMLfile with several quan <div>s
quanTitle=[] #Title for Quan first level of div
xmlLevel=0 #div level
pageNumber=1 #page number
lineNumber=1 #line number line8 add PageAB line16 add PageNumber
#lineBreak=0 #whether the line is broken
hi=re.compile(r"　([^\s　<>]+)\s([^\s　<>]*?)　", re.A)
duiyuzheju=re.compile(r"\A(　?)(對語|摘句)", re.A)
#hi2=re.compile(r"　([^\s\n　<>]+)\s([^\s\n　<>]*?)$", re.A)
item=re.compile(r"\A　?(.)　?<hi>(..切)([^<]*</hi>)", re.A)
for para in document.readlines(): #deal with each paragraph
    if para[0:2]=="##": # test whether begins with "##"
        pageTitle=para.split("/")
        if len(pageTitle)==1: #if only "##" => end of file
            break
        quan=pageTitle[len(pageTitle)-1]
        lineNumber=0 #set lineNumber to 0
        if not (quan in quanTitle): #test whether it is a new quan title
            quanTitle.append(quan) #add the title into the quan title list
            if not xmlLevel==0:
                newDoc = newDoc+ "<pb n='"+str(pageNumber)+"b'/>" + "</p>"+"</div>" * xmlLevel
            pageNumber = 1
            newDoc = newDoc+ "\n<div>"+"<head>" + quan + "</head>\n<p>" #add in new quan title
            xmlLevel = 1
            duiyu=0
        else:
            newDoc = newDoc+ "<pb n='"+str(pageNumber)+"b'/>\n"
            pageNumber += 1
    else:
        if not len(para)==0: # not empty line
            #if lineBreak == 0:
            text=para
            while re.search(hi, text):
                text = re.sub(hi, r"<hi>\1\2</hi>", text) # hi
            if re.search(duiyuzheju, text):
                if xmlLevel==4:
                    text=re.sub(duiyuzheju, r"</p>\n</div><div><head>\2</head><p>", text)
                elif xmlLevel==3:
                    text=re.sub(duiyuzheju, r"</p>\n<div><head>\2</head><p>", text)
                xmlLevel = 4
            elif re.match(r"\A　　　?([^　]{1,2}聲)", text): #heading level 2
                if xmlLevel==2:
                    text = re.sub(r"\A　　　?([^　])", r"</p></div>\n<div><head>\1", text)
                elif xmlLevel==1:
                    xmlLevel=2
                    text = re.sub(r"\A　　　?([^　])", r"</p><div><head>\1", text)
                elif xmlLevel==3:
                    xmlLevel=2
                    text = re.sub(r"\A　　　?([^　])", r"</p></div></div>\n<div><head>\1", text)
                elif xmlLevel==4:
                    xmlLevel=2
                    text = re.sub(r"\A　　　?([^　])", r"</p></div></div></div>\n<div><head>\1", text)
                text += "</head>\n<p>"
            elif re.match(item, text):
                if xmlLevel==2:
                    text=re.sub(item, r"</p>\n<div><head>\1<hi><span n='fanqie'>\2</span>\3</head><p>", text)
                elif xmlLevel==3:
                    text=re.sub(item, r"</p></div>\n<div><head>\1<hi><span n='fanqie'>\2</span>\3</head><p>", text)
                elif xmlLevel==4:
                    text=re.sub(item, r"</p></div></div>\n<div><head>\1<hi><span n='fanqie'>\2</span>\3</head><p>", text)
                xmlLevel=3
            elif re.match(r"\A　?　?(韻[藻澡]|[增増])[^　]{2,}", text):
                if xmlLevel==3:
                    text=re.sub(r"\A　?　?(韻[藻澡]|[增増])", r"</p>\n<div><head>\1</head><p>", text)
                elif xmlLevel==4:
                    text=re.sub(r"\A　?　?(韻[藻澡]|[增増])", r"</p></div>\n<div><head>\1</head><p>", text)
                if re.search(r"<p>([增増])([^　]{2,})", text):
                    text=re.sub(r"<p>([增増])([^　]{2,})", r"</div><div><head>\1</head><p>\2", text)
                xmlLevel=4
            newDoc += text
        lineNumber += 1
        if lineNumber == 9:
            newDoc += "<pb n='"+str(pageNumber)+"a'/>"
newDoc += "</p>\n" + "</div>" * (xmlLevel) + "</body></text></TEI>"
f = open("/Users/pingtzuhu/Documents/佩文韻府/test.xml", "w", encoding="utf-8")
f.write(newDoc)
f.close
print ("We're done.")

We're done.
